<a href="https://colab.research.google.com/github/prayanshgupta129/Wallet-Risk-Scoring-From-Scratch/blob/main/Wallet_Risk_Scoring_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install subgrounds gql requests pandas numpy scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.26.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.25.2 which is incompatible.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.25.2 which is incompatible.


In [24]:
from subgrounds import Subgrounds
import pandas as pd

# === Step 1: Connect to Subgraph ===
sg = Subgrounds()
subgraph = sg.load_subgraph("https://gateway.thegraph.com/api/23ad967061b76af0136f8dd9942c24ea/subgraphs/id/AwoxEZbiWLvv6e3QdvdMZw4WDURdGbvPfHmZRc8Dpfz9")

# === Step 2: Define Entities ===
deposits = subgraph.Query.deposits
borrows = subgraph.Query.borrows
repays = subgraph.Query.repays

# === Step 3: Fetch Data (First 5000 records per entity) ===
df_deposits = sg.query_df([
    deposits.account.id,
    deposits.amount
])
df_deposits.rename(columns={"deposits_account_id": "wallet_id", "deposits_amount": "amount"}, inplace=True)

df_borrows = sg.query_df([
    borrows.account.id,
    borrows.amount
])
df_borrows.rename(columns={"borrows_account_id": "wallet_id", "borrows_amount": "amount"}, inplace=True)

df_repays = sg.query_df([
    repays.account.id,
    repays.amount
])
df_repays.rename(columns={"repays_account_id": "wallet_id", "repays_amount": "amount"}, inplace=True)

# === Step 4: Extract active wallets ===
wallet_list = df_deposits['wallet_id'].unique().tolist()
print(f"Using {len(wallet_list)} active wallets from subgraph")

# === Step 5: Compute credit scores ===
results = []
for wallet in wallet_list:
    total_deposited = df_deposits[df_deposits["wallet_id"] == wallet]["amount"].sum()
    total_borrowed = df_borrows[df_borrows["wallet_id"] == wallet]["amount"].sum()
    total_repaid   = df_repays[df_repays["wallet_id"] == wallet]["amount"].sum()

    # Credit Score Logic
    if total_borrowed == 0:
        credit_score = 0.0
    else:
        repay_ratio = total_repaid / total_borrowed
        credit_score = round(min(1.0, repay_ratio) * 1000, 2)

    results.append({
        "wallet": wallet,
        "credit_score": credit_score
    })

# === Step 6: Export or Display ===
df_results = pd.DataFrame(results)
print(df_results.head(10))  # show first 10

# Save to CSV
df_results.to_csv("credit_scores.csv", index=False)














Using 90 active wallets from subgraph
                                       wallet  credit_score
0  0x84e2c0837d761ef6078b9dbf3839e108155d84f2           0.0
1  0x7f7bbb19026b5eca0b6cdb96caa84923716532a5           0.0
2  0x7aa50cb10a59dc1d5be57b3a4010d253811d0ea2           0.0
3  0xf007d777a761fb94f29193e902fca80181c8b009           0.0
4  0xbb01bfa9171ca3f3acf86393bf51e871812d1684           0.0
5  0x1f72aeb2407ab1832ede569b8aa49866aafa0f10           0.0
6  0x9c483fa4d84a218940bfa02652fcaf01ee8f44f2           0.0
7  0x7f3bd73c598b5956440089fda4e99c950feb6a83           0.0
8  0xa3e5e60b6d5f23aebc91a062b96ea3cad1a341f0           0.0
9  0x4303c46c71f6e48eaa96ab3f194b79fac082e46a           0.0
